In [2]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.tools.sql_database.tool import InfoSQLDatabaseTool
from langchain.prompts import SystemMessagePromptTemplate, MessagesPlaceholder
from langchain.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langsmith import Client
client = Client()

In [6]:
def get_db_engine(db_path):
    """로컬 SQLite DB 파일과 연결된 엔진을 생성합니다."""
    try:
        # SQLite DB 파일과 연결
        connection = sqlite3.connect(db_path, check_same_thread=False)
        
        # SQLAlchemy 엔진 생성
        engine = create_engine(
            f"sqlite:///{db_path}",
            poolclass=StaticPool,
            connect_args={"check_same_thread": False}
        )
        
        return engine
    
    except Exception as e:
        print(f"데이터베이스 연결 중 오류 발생: {str(e)}")
        return None

# DB 파일 경로 지정
db_path = './data/real_estate.db'

# 엔진 생성
engine = get_db_engine(db_path)

# LangChain SQLDatabase 객체 생성
db = SQLDatabase(engine)

In [1]:
pip install -U langsmith openai

  Using cached openai-1.58.1-py3-none-any.whl.metadata (27 kB)
Using cached openai-1.58.1-py3-none-any.whl (454 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.52.1
    Uninstalling openai-1.52.1:
      Successfully uninstalled openai-1.52.1
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.136
    Uninstalling langsmith-0.1.136:
      Successfully uninstalled langsmith-0.1.136
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.4 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.2.6 which is incompatible.
langchain-community 0.3.3 requires langsmith<0.2.0,>=0.1.125, but you have langsmith 0.2.6 which is incompatible.


In [ ]:
property_rental = db.get_table_info(table_names=["property_rentals", "property_info", "property_locations"])

    rental_prompt = """
    질문:
    {questions}

    사용 가능한 테이블 스키마:
    {property_rental}

    당신은 SQL 전문가입니다. 
    데이터베이스는 {dialect} 방언을 사용합니다.

    다음 규칙을 따라주세요:
    1. 한국어로 응답해주세요.
    2. SQL 쿼리를 작성할 때는 명확하고 효율적이어야 합니다.
    3. 결과는 최대 {top_k}개까지만 보여주세요.
    4. 금액에 관련된 쿼리를 작성할 때는 쉼표(,)를 제거하고 숫자로 변환해야 합니다.
    5. 에러가 발생하면 원인을 설명하고 수정된 쿼리를 제시해주세요.
    6. rental_type이 30051B1는 전세입니다.
    7. rental_type이 30051B2는 월세입니다.

    SELECT 
        pl.sigungu,
        pi.description,
        pr.deposit as "보증금(만원)",
        pl.latitude,
        pl.longitude
    FROM property_rentals pr
    JOIN property_info pi ON pr.property_id = pi.property_id
    JOIN property_locations pl ON pi.location_id = pl.location_id
    WHERE pl.sigungu LIKE '%강남구%'
    AND pr.rental_type = '30051B1'
    AND CAST(REPLACE(pr.deposit, ',', '') AS DECIMAL) <= 10000;

    이런식으로 쿼리를 짜줘.
    """

    prompt=rental_prompt.format(
        dialect="SQLite",
        top_k=5, 
        property_rental=property_rental,
        questions=state["questions"]
    )

    response = llm(messages=[{"role": "system", "content": prompt}])

    llm_query = response.content

    tool = QuerySQLDataBaseTool(db=db)

    # SQL 쿼리 실행"
    results = tool._run(llm_query)

In [7]:
# Step 1: LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# Step 2: 필터링 (부동산 관련 여부 판단)
filter_prompt = PromptTemplate(
    input_variables=["question"],
    template=
    """당신은 부동산 관련 질문을 분류하는 전문가입니다.
주어진 질문이 부동산(매매, 전세, 월세, 가격, 위치, 면적 등)과 관련된 것인지 판단해주세요.

질문: "{question}"

규칙:
1. 다음과 관련된 질문은 "YES"로 답변:
   - 부동산 매매/임대 거래
   - 집값, 전세가, 월세
   - 아파트, 주택 정보
   - 지역별 부동산 정보
   - 부동산 시세
   - 건물 면적, 층수

2. 다음은 "NO"로 답변:
   - 날씨, 음식 등 부동산과 무관한 주제
   - 일반적인 잡담

반드시 "YES" 또는 "NO"로만 답변하세요.

예시:
- "강남구 아파트 전세 얼마야?" -> "YES"
- "서울시 월세 시세 알려줘" -> "YES"
- "아파트 가격이 어떻게 되나요?" -> "YES"
- "오늘 날씨 어때?" -> "NO"
- "점심 뭐 먹을까?" -> "NO"

그리고 만약에 답변이 "NO"라면 부동산 관련 질문이 아닌 이유를 말하시오
답변:
"""
)

def is_real_estate_related(question: str) -> bool:
    response = llm.invoke(filter_prompt.format(question=question))  # __call__ 대신 invoke 사용
    content = response.content.upper()  # 대소문자 구분 없이 체크
    return "YES" in content

db = SQLDatabase(
    engine,
    sample_rows_in_table_info=False  # 샘플 행 조회 비활성화
)

# 나머지 코드는 동일
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
db_info = db.get_table_info()

custom_system_prompt = """
사용 가능한 테이블 스키마:
{db_info}

테이블 스키마 상세정보:

- 테이블 이름: `real_estate_info`
  * property_id: 부동산 ID (INTEGER, PK)
  * location_id: 위치 ID (INTEGER, FK)
  * property_type: 부동산 유형 (VARCHAR(20))
  * building_name: 건물 이름 (VARCHAR(100))
  * construction_year: 건축 연도 (INTEGER)
  * floor: 층수 (INTEGE)
  * exclusive_area: 전용 면적 (DECIMAL(10,2))

- 테이블 이름: `location`
  * location_id: 위치 ID (INTEGER, PK)
  * sido: 시/도 (VARCHAR(20))
  * sigungu: 시/군/구 (VARCHAR(20))
  * dong: 동 (VARCHAR(20))
  * road_name: 도로명 주소 (VARCHAR(100))
  * jibun: 지번 주소 (VARCHAR(20))
  * latitude: 위도 (DECIMAL(10,7))
  * longitude: 경도 (DECIMAL(10,7))

- 테이블 이름: `sale_transaction`
  * sale_id: 매매 거래 ID (INTEGER, PK)
  * property_id: 부동산 ID (INTEGER, FK)
  * transaction_date: 거래 날짜 (DATE)
  * price: 매매 가격(만원) (DECIMAL(15,0))
  * transaction_type: 거래 유형 (VARCHAR(20))
  * broker_location: 중개사 위치 (VARCHAR(100))

- 테이블 이름: `rental_transaction`
  * rental_id: 임대 거래 ID (INTEGER, PK)
  * property_id: 부동산 ID (INTEGER, FK)
  * rental_type: 임대 유형 (VARCHAR(10))
  * contract_date: 계약 날짜 (DATE)
  * deposit: 보증금(만원) (DECIMAL(15,0))
  * monthly_rent: 월세(만원) (DECIMAL(10,0))
  * contract_period: 계약 기간 (VARCHAR(50))


당신은 SQL 전문가입니다. 
데이터베이스는 {dialect} 방언을 사용합니다.

다음 규칙을 따라주세요:
1. 한국어로 응답해주세요.
2. SQL 쿼리를 작성할 때는 명확하고 효율적이어야 합니다.
3. 결과는 최대 {top_k}개까지만 보여주세요.
4. 금액에 관련된 쿼리를 작성할 때는 쉼표(,)를 제거하고 숫자로 변환해야 합니다.
5. 에러가 발생하면 원인을 설명하고 수정된 쿼리를 제시해주세요.

예를 들어서 사용자 응답이 '서울시 강남구 1억 이하 전세를 추천해줘'라고 온다면

SELECT 
    l.sigungu,
    r.building_name, # 없으면 r.property_type
    rt.deposit as "보증금(만원)",
    r.exclusive_area as "전용면적(㎡)",
FROM rental_transaction rt
JOIN real_estate_info r ON rt.property_id = r.property_id
JOIN location l ON r.location_id = l.location_id
WHERE l.sigungu LIKE '%강남구%'
AND rt.rental_type = '전세'
AND CAST(REPLACE(rt.deposit, ',', '') AS DECIMAL) <= 10000;

이런식으로 쿼리를 짜서 나온 값 중에
너가 생각하기에 추천을 할 만할 매물들을 5개 추천해줘
그리고 답변시 건물명이 없으면 property_type이 뭔지 알려줘

그리고 답변하기 전에 이 답이 맞는지 확인해주세요.
"""

# Step 4: SQL 설정
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(custom_system_prompt),
    MessagesPlaceholder(variable_name="messages"),
])

system_message = prompt_template.format(dialect="SQLite", top_k=5, messages=[], db_info=db_info)

agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)
# 테스트
if __name__ == "__main__":
    question = input("저는 부동산 챗봇입니다! 질문해주세요: ")

    try:
        if not is_real_estate_related(question):
            print("부동산 관련 질문이 아닙니다. 다른 질문을 해주세요.")
        else:
            events = agent_executor.stream(
                {"messages": [("user", question)]},
                config={"recursion_limit": 50},
                
                stream_mode="values",
            )

            for event in events:
                event["messages"][-1].pretty_print()
    except Exception as e:
        print(f"오류 발생: {str(e)}")

================================ Human Message =================================

강남구 집값 저렴한 곳을 알려줘
================================== Ai Message ==================================
Tool Calls:
  sql_db_query_checker (call_OcahcR99GE0QWlpo0Spomqhv)
 Call ID: call_OcahcR99GE0QWlpo0Spomqhv
  Args:
    query: SELECT 
    l.sigungu,
    r.building_name, 
    s.price as "가격(만원)",
    r.exclusive_area as "전용면적(㎡)"
FROM sale_transaction s
JOIN real_estate_info r ON s.property_id = r.property_id
JOIN location l ON r.location_id = l.location_id
WHERE l.sigungu LIKE '%강남구%'
ORDER BY s.price ASC
LIMIT 5;
================================= Tool Message =================================
Name: sql_db_query_checker

```sql
SELECT 
    l.sigungu,
    r.building_name, 
    s.price as "가격(만원)",
    r.exclusive_area as "전용면적(㎡)"
FROM sale_transaction s
JOIN real_estate_info r ON s.property_id = r.property_id
JOIN location l ON r.location_id = l.location_id
WHERE l.sigungu LIKE '%강남구%'
ORDER BY s.price ASC


In [14]:
db_info = db.get_table_info()
info_tool = InfoSQLDatabaseTool(db=db)
schema_info = info_tool.run("rental_transaction")

In [34]:
print(schema_info)


CREATE TABLE rental_transaction (
	rental_id INTEGER, 
	property_id INTEGER, 
	rental_type VARCHAR(10), 
	contract_date DATE, 
	deposit DECIMAL(15, 0), 
	monthly_rent DECIMAL(10, 0), 
	contract_period VARCHAR(50), 
	PRIMARY KEY (rental_id), 
	FOREIGN KEY(property_id) REFERENCES real_estate_info (property_id)
)


In [18]:
query = """
SELECT 
    l.sigungu,
    l.road_name,
    r.building_name, 
    rt.monthly_rent as "월세(만원)",
    r.exclusive_area as "전용면적(㎡)",
    r.property_type as "건물유형"
FROM rental_transaction rt
JOIN real_estate_info r ON rt.property_id = r.property_id
JOIN location l ON r.location_id = l.location_id
WHERE l.sigungu LIKE '%금천구%'
AND rt.rental_type = '월세'
AND CAST(REPLACE(rt.monthly_rent, ',', '') AS DECIMAL) <= 50;

"""

result = db.run(query)
print("DB 조회 결과:")
print(result)

DB 조회 결과:
[('금천구', '독산로64다길', '', 20, 82.68, '단독다가구'), ('금천구', '벚꽃로50길', '', 35, 23, '단독다가구'), ('금천구', '독산로106길', '', 35, 25, '단독다가구'), ('금천구', '독산로84길', '', 10, 65.12, '단독다가구'), ('금천구', '독산로70라길', '', 30, 20, '단독다가구'), ('금천구', '독산로', '', 50, 50, '단독다가구'), ('금천구', '독산로', '', 20, 30, '단독다가구'), ('금천구', '벚꽃로24길', '', 22, 10, '단독다가구'), ('금천구', '남부순환로128길', '', 38, 19, '단독다가구'), ('금천구', '문성로3길', '', 50, 44.61, '단독다가구'), ('금천구', '시흥대로147길', '', 34, 15, '단독다가구'), ('금천구', '독산로84길', '', 30, 20, '단독다가구'), ('금천구', '독산로51길', '', 20, 25, '단독다가구'), ('금천구', '탑골로2길', '', 30, 29.72, '단독다가구'), ('금천구', '독산로78가길', '', 27, 15.7, '단독다가구'), ('금천구', '독산로60길', '', 40, 39.6, '단독다가구'), ('금천구', '벚꽃로42길', '', 40, 19.02, '단독다가구'), ('금천구', '금하로', '', 50, 47.43, '단독다가구'), ('금천구', '시흥대로84라길', '', 40, 33.05, '단독다가구'), ('금천구', '독산로87길', '', 40, 30, '단독다가구'), ('금천구', '독산로94길', '', 15, 43.38, '단독다가구'), ('금천구', '독산로72길', '', 20, 20.05, '단독다가구'), ('금천구', '시흥대로72길', '', 47, 18, '단독다가구'), ('금천구', '독산로24길', '', 20, 84.42, '단독다

In [46]:
type(result)

str